In [108]:
import requests
import zipfile
import pandas as pd
import numpy as np
import time
import timeit
from zipfile import ZipFile
from StringIO import StringIO
import urllib2
import fnmatch
import os
#import boto3
#import botocore
import string
from oca_utils.oca_utils import *

# PreTrial Data Extract

This is the standalone file to extract the pretrial data from NYS OCA. 
This script can pull the data from OCA (release every 6 months) and upload it to the BxD Database for analytics use. 

This is written in python 2.7 for cross compatibility with the BxD ec2 instance. 

In [109]:
today_str = time.strftime("%Y-%m-%d")
link_url= "http://www.nycourts.gov/"
link_subpath= "LegacyPDFS/court-research/"
link_file= "PretrialReleaseDataExtractWeb2020.zip"
link_url= link_url+link_subpath+link_file

In [110]:
# Try link for downloadable content - based on header information
try:
    is_downloadable(link_url)
    print("[OCA STAT] - Vaild URL. Reading data.")
except requests.ConnectionError as exception:
    print("[OCA STAT] - Not a valid URL.")
    

[OCA STAT] - Vaild URL. Reading data.


#### Quick load

set the `quick_load` to True, to skip the url request and simply read the most recent `.csv` extract. 

In [111]:
quick_load = False
if quick_load:
    pretrial = pd.read_csv("PretrialReleaseDataExtractWeb2020-revised.csv")

The link works. Will extract from zip file on OCA site and read the output csv. 

In [112]:
start_time = timeit.default_timer()

r = urllib2.urlopen(link_url).read()
file_ex = ZipFile(StringIO(r))
file_ex.extractall()
pattern = "*-revisedWO.csv"

for root, dirs, files in os.walk(os.getcwd(),):
    for filename in fnmatch.filter(files, pattern):
        pretrial_csv = file_ex.open(filename)
        pretrial = pd.read_csv(pretrial_csv)

elapsed = timeit.default_timer() - start_time
print(elapsed)

45.81822896


#### Tested on EC2

The above extract does work on the. BxD ec2 instance as of 7/24/2021 around 12:00PM EST. 

## Data Transformation

Now we turn our attention to transforming and cleaning up the data in the file. We will test several operations on both here and on ec2, so that they we are sure it can work in the cloud. Given that we have run into issues previously with instance timeout and memory kills, hopefully this approach will side-step those issues. 

#### Date Format

#### Test on EC2

This process take a large amount of time - 90 seconds on ec2 and close to a minute here. As much as it is desirable to have a consistent date format completed on the back-end, it might be worth exploring ThoughtSpot's ability to transform these date types if this code chunk becomes too large to complete on a t2.micro. The column `rearrest_date` is of the format "Mar2021" and need to be reassigned to datetime format. 

**OCA continues to change the file format so this may change.** They do seem to correct the date fields to a standard format, which is nice. `rearrest_date` may be correct in the future. 

In [113]:
pretrial.columns = pretrial.columns.str.lower()
pretrial.columns = pretrial.columns.str.replace(" ", "_")
pretrial.columns = pretrial.columns.str.replace("-", "_")
date_cols = [col for col in pretrial.columns if col.endswith('_date')]
not_date_type = ["rearrest_date"]
for col in date_cols:
    if col in not_date_type:
        pretrial[col] = pd.to_datetime(pretrial[col],format="%b%Y") # fix Mar2020 format type
    pretrial[col] = pd.to_datetime(pretrial[col],errors='coerce') # assign datetime to all else

In [114]:
top_columns = [col for col in pretrial.columns if col.startswith("top_")]
top_charges = [col for col in pretrial if col.startswith("top_charge_at") and not col.endswith("_ind")]

In [115]:
top_strings = pretrial[["top_arrest_article_section", "top_charge_at_arrest",
                        "top_charge_at_conviction","top_conviction_article_section",
                        "top_charge_at_arraign", "top_arraign_article_section",]]

#### Blank top_xx_article_section

1) \_arrest: What to do this theese? Leave blank or convert to arraignment charges. There are only 2457 (1.3% of total data)

2) \_arraign: Same question but less often. only 0.76% (1406 total) in the data set. 

3) \_conviction: Much dirter entries. Some are NaN (expected) but the formatting is odd. some are dates, many dashes and letters that don't exist in the other article_section columns. The very sketchy ones look to be very very rare. 

#### Tested on ec2 
lowering works for all columns in this method. loop is better than .applymap <br>
- changed oca_stat.py to reflect this new learning. As of 7/26/2021

In [116]:
start_time = timeit.default_timer()

for col in pretrial:
    if pretrial[col].dtype == "O":
        pretrial[col] = pretrial[col].str.lower()
        pretrial[col] = pretrial[col].str.replace("-","")
        pretrial[col] = pretrial[col].str.replace("\(", "").str.replace("\)","")

        
elapsed = timeit.default_timer() - start_time
print "time to complete string manipulation: ", elapsed

time to complete string manipulation:  31.1439847946


### New Columns - article, section, subsection

There are 105 columns in this data set, that's a lot. We have made more in other data set to separate out article and section for top charge at arraignment. It's helpful in ThoughtSpot to have multiple ways to filter so that TS isn't filtering everything via the same column. 

In [117]:
top_strings.columns

Index([u'top_arrest_article_section', u'top_charge_at_arrest',
       u'top_charge_at_conviction', u'top_conviction_article_section',
       u'top_charge_at_arraign', u'top_arraign_article_section'],
      dtype='object')

In [118]:
arraign = pretrial["top_charge_at_arraign"].str.extract(r'((\d+) (\w+) (\w+))') \
    .rename(columns = {0:"full", 1:"section",2:"subsection",3:"charge"})
arrest = pretrial["top_charge_at_arrest"].str.extract(r'((\d+) (\w+) (\w+))') \
    .rename(columns = {0:"full", 1:"section",2:"subsection",3:"charge"})
convict = pretrial["top_charge_at_conviction"].str.extract(r'((\d+) (\w+) (\w+.{3}))') \
    .rename(columns = {0:"full", 1:"section",2:"subsection",3:"charge"})

/Users/jeffshamp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/Users/jeffshamp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/Users/jeffshamp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [119]:
pretrial["arraign_subsection_bxd"] = arraign['subsection']
pretrial["arrest_subsection_bxd"] = arrest['subsection']
pretrial["conviction_subsection_bxd"] = convict['subsection']

In [121]:
weights = pretrial.top_charge_weight_at_arraign.unique()

In [122]:
pretrial.arraign_subsection_bxd = pretrial.arraign_subsection_bxd.apply(lambda x:clean_subsection(x, weights))
pretrial.arrest_subsection_bxd = pretrial.arrest_subsection_bxd.apply(lambda x:clean_subsection(x, weights))
pretrial.conviction_subsection_bxd = pretrial.conviction_subsection_bxd.apply(lambda x:clean_subsection(x, weights))

In [123]:
pretrial[["arraign_article_bxd", "arraign_section_bxd"]] = \
            pretrial.top_arraign_article_section.str.split(".",n=1,expand=True)
pretrial[["arrest_article_bxd", "arrest_section_bxd"]] = \
            pretrial.top_arraign_article_section.str.split(".",n=1,expand=True)
pretrial[["conviction_article_bxd", "conviction_section_bxd"]] = \
            pretrial.top_arraign_article_section.str.split(".",n=1,expand=True)

## Save cleaned data

In [47]:
save_clean = False
if save_clean:
    pretrial.to_csv("pretrial_clean.csv")
load_clean = False
if load_clean: # fix date type inference
    pretrial = pd.read_csv("pretrial_clean.csv")

## Judges

There are two strange judge name types. "Office, Clerk's" and "judege/jho/hearing examiner, visiting". These types make up ~3000 judge name entries, so it's fairly common in the dataset. They usully show up in upstate counties as seen below

In [48]:
pretrial[pretrial.judge_name.str.contains("office|hearing")] \
    .groupby("county_name").count()["court_name"] \
    .sort_values(ascending=False)[:15]

county_name
onondaga       1016
oneida          439
ontario         268
steuben         202
cayuga          140
albany          127
cattaraugus      93
warren           90
oswego           59
bronx            57
erie             56
clinton          50
ulster           25
niagara          20
westchester      15
Name: court_name, dtype: int64

In [49]:
pretrial.judge_name.describe()

count              184118
unique                594
top       graf,_alfred_c.
freq                 2987
Name: judge_name, dtype: object

## Bail

Not sure if we care about the ratio of cash to credit in bail amounts, but it does seem to be used in a weird way. Bascially, no one uses credit even if it is less in amount than cash. 

73% of bail is not paid, 14% is paid in cash, 11% bond, and 1% is credit. 

In [17]:
#pretrial['cash_credit_ratio_bail'] = pretrial.first_bail_set_cash/pretrial.first_bail_set_credit

In [124]:
bail_set = pretrial[(pretrial.first_bail_set_cash.isnull() == False) |
        (pretrial.first_bail_set_credit.isnull() == False)].shape[0]

In [125]:
pretrial[(pretrial.first_bail_set_cash.isnull() == False) |
        (pretrial.first_bail_set_credit.isnull() == False)] \
            .bail_made_indicator.value_counts(dropna=False)/bail_set*100

NaN       73.352360
cash      14.073418
bond      11.557261
credit     1.016960
Name: bail_made_indicator, dtype: float64

### New Columns - bail made amount - dollar bail

New column for bail made amount. If bail is paid, then this column determines the method of payment and records that amount. If bail is set at 5000:cash, 1000:credit, and 6000:bond, and a bond is taken, then 6000 will be recorded in `bail_made_amount_bxd`.

`dollar_bail_bxd` tracks all the `first_bail_set_cash` values for those equal to 1\$. The cash bail set field seesm to dicated the dollar bail cases. 

In [126]:
cash_ser = pretrial[pretrial.bail_made_indicator == "cash"].first_bail_set_cash
credit_ser = pretrial[pretrial.bail_made_indicator == "credit"].first_bail_set_credit
bond_ser = pretrial[pretrial.bail_made_indicator == "bond"].first_insurance_company_bail_bond
null_ser = pretrial[pretrial.bail_made_indicator.isnull()].bail_made_indicator
bail_amount = pd.concat([cash_ser, credit_ser, bond_ser, null_ser]) \
        .rename("bail_made_amount_bxd")
pretrial = pretrial.join(bail_amount)

In [127]:
pretrial['dollar_bail_bxd'] = False
pretrial.loc[(pretrial.first_bail_set_cash == 1.0),
             'dollar_bail_bxd'] = True

## Priors

### New columns - prior conviction - pending arrests

In [128]:
priors = [col for col in pretrial.columns if col.startswith("prior")]
pending = [col for col in pretrial.columns if col.startswith("pend_")]

In [129]:
pretrial["priors_sum_bxd"] = pretrial[priors].sum(axis=1)
pretrial["pending_arrest_sum_bxd"] = pretrial[pending].sum(axis=1)

## Upload to s3

In [ ]:
print("[OCA STAT] - Saving to s3 {}" \
          .format(time.strftime("%Y-%m-%d:%H:%M:%S")))
########### Upload to S3 ######################
    
tgt_bucket = 'bxdgoogletransformed'
tgt_csv = 'pretrial'
s3 = boto3.resource('s3')

f = open(tgt_csv+'.csv', 'rb')
try:
    s3.Bucket(tgt_bucket).put_object(
                Key='extract_'+today_str+'/'+tgt_csv+'.csv',
                ServerSideEncryption='aws:kms',
                StorageClass='STANDARD_IA',
                Body=f)
except:
    print("[OCA STAT] - s3 resource error")

print("[OCA STAT] - Done {}" \
      .format(time.strftime("%Y-%m-%d %H:%M:%S")))